In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.feature_selection import chi2, f_regression, f_classif, mutual_info_classif, mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel, VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn import model_selection as ms
from sklearn.model_selection import cross_val_score, cross_val_predict

sklearn.set_config(print_changed_only=False)

import xgboost as xgb

from matplotlib import pyplot as plt

import import_ipynb
from Self_Written_Functions_Sheet import nan_count, nan_col_obj_list_fillna
from Self_Written_Functions_Sheet import mvarea_1_to_0, mvarea_0_to_mean, mvtype_none_to_mode
from Self_Written_Functions_Sheet import rmse

importing Jupyter notebook from Self_Written_Functions_Sheet.ipynb


In [2]:
# import dataset

hp_validation = pd.read_csv('../test.csv')
hp_validation

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,Comp

In [3]:
hp_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [4]:
hp_validation.isnull().sum()

Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64

In [5]:
hp_validation.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,1458.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1457.000000,1457.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.00000,1381.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,554.294925,1046.117970,1156.534613,325.967786,3.543523,1486.045922,0.434454,0.065202,1.570939,0.377656,2.854010,1.042495,6.385195,0.58122,1977.721217,1.766118,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,176.753926,437.260486,442.898624,398.165820,420.610226,44.043251,485.566099,0.530648,0.252468,0.555190,0.503017,0.829788,0.208472,1.508895,0.64742,26.431175,0.775945,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,407.000000,0.000000,0.000000,407.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.00000,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,219.250000,784.000000,873.500000,0.000000,0.000000,1117.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.00000,1959.000000,1.000000,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,0.000000,460.000000,988.000000,1079.000000,0.000000,0.000000,1432.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,0.00000,1979.000000,2.000000,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,0.000000,797.750000,1305.000000,1382.500000,676.000000,0.000000,1721.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.00000,2002.000000,2.000000,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,2140.000000,5095.000000,5095.000000,1862.000000,1064.000000,5095.000000,3.000000,2.000000,4.000000,2.000000,6.000000,2.000000,15.000000,4.00000,2207.000000,5.000000,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


# Basic EDA and Imputations

In [6]:
# adding the number of bathrooms
hp_validation['TotalBaths'] = hp_validation['FullBath'] + hp_validation['HalfBath'] * 0.5

# adding the number of bathrooms in the basement
hp_validation['BsmtBaths'] = hp_validation['BsmtFullBath'] + hp_validation['BsmtHalfBath'] * 0.5

# adding the total number of finished SF of the basement
hp_validation['BsmtFinTotalSF'] = hp_validation['BsmtFinSF1'] + hp_validation['BsmtFinSF2']

# determining if there is unfinished SF in the basement
hp_validation['BsmtUnfin'] = [1 if x > 0 else 0 for x in hp_validation['BsmtUnfSF']]

# determining if a house has a 2nd floor
hp_validation['2ndFlr'] = [1 if x > 0 else 0 for x in hp_validation['2ndFlrSF']]

# determining if a house has any low quality finishes
hp_validation['LowQualFin'] = [1 if x > 0 else 0 for x in hp_validation['LowQualFinSF']]


# https://www.fixr.com/comparisons/porch-vs-deck#cQ
    # decks are diff than porches, but it seems that porches can be grouped
# adding the total SF of porches
hp_validation['TotalPorchSF'] = hp_validation['OpenPorchSF'] + hp_validation['EnclosedPorch'] + hp_validation['3SsnPorch'] + hp_validation['ScreenPorch']

# determining if there are any porches at all
hp_validation['Porches'] = [1 if x > 0 else 0 for x in hp_validation['TotalPorchSF']]

# determining if there are any decks at all
hp_validation['Decks'] = [1 if x > 0 else 0 for x in hp_validation['WoodDeckSF']]

# calculating the time between remodeling and selling the house
hp_validation['YrsRemodToSold'] = hp_validation['YrSold'] - hp_validation['YearRemodAdd']

# dropping some columns bc there are too many null values
hp_val = hp_validation.drop(['Alley',
                             'PoolQC',
                             'Fence',
                             'MiscFeature'], axis=1).copy()

hp_val

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,0,6,2010,WD,Normal,1.0,0.0,612.0,1,0,0,120,1,1,49
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal,1.5,0.0,923.0,1,0,0,36,1,1,52
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,0,3,2010,WD,Normal,2.5,0.0,791.0,1,1,0,34,1,1,12
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,0,6,2010,WD,Normal,2.5,0.0,602.0,1,1,0,36,1,1,12
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,0,1,2010,WD,Normal,2.0,0.0,263.0,1,0,0,226,1,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,0,6,2006,WD,Normal,1.5,0.0,0.0,1,1,0,0,0,0,36
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,0,4,2006,WD,Abnorml,1.5,0.0,252.0,1,1,0,24,1,0,36
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960

In [7]:
hp_val_xgb = hp_validation.copy()
hp_val_xgb.to_csv('..\hp_val_xgb.csv')

In [8]:
nan_count(hp_val)

,Null_Count
MSZoning,4
LotFrontage,227
Utilities,2
Exterior1st,1
Exterior2nd,1
MasVnrType,16
MasVnrArea,15
BsmtQual,44
BsmtCond,45
BsmtExposure,44


In [9]:
hp_val['MasVnrType'].value_counts()

None       878
BrkFace    434
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64

In [10]:
# checking null values as dataframe
hp_val[hp_val['MasVnrType'].isnull()][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
231,NaN,NaN
246,NaN,NaN
422,NaN,NaN
532,NaN,NaN
544,NaN,NaN
581,NaN,NaN
851,NaN,NaN
865,NaN,NaN
880,NaN,NaN
889,NaN,NaN


In [11]:
hp_val[(hp_val['MasVnrType']=='None') & (hp_val['MasVnrArea']>0)][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
209,None,285.0
992,None,1.0


In [12]:
# running functions from Self_Written_Functions_Sheet to fill MasVnrType/Area
mvarea_1_to_0(hp_val)
mvarea_0_to_mean(hp_val)
mvtype_none_to_mode(hp_val)

In [13]:
hp_val

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold
0,1461,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,0,6,2010,WD,Normal,1.0,0.0,612.0,1,0,0,120,1,1,49
1,1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal,1.5,0.0,923.0,1,0,0,36,1,1,52
2,1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,0,3,2010,WD,Normal,2.5,0.0,791.0,1,1,0,34,1,1,12
3,1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,0,6,2010,WD,Normal,2.5,0.0,602.0,1,1,0,36,1,1,12
4,1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,0,1,2010,WD,Normal,2.0,0.0,263.0,1,0,0,226,1,0,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,0,6,2006,WD,Normal,1.5,0.0,0.0,1,1,0,0,0,0,36
1455,2916,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,0,4,2006,WD,Abnorml,1.5,0.0,252.0,1,1,0,24,1,0,36
1456,2917,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960

In [14]:
hp_val['MasVnrType'].value_counts()

None       877
BrkFace    436
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64

In [15]:
hp_val[hp_val['MasVnrType'].isnull()][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
231,NaN,NaN
246,NaN,NaN
422,NaN,NaN
532,NaN,NaN
544,NaN,NaN
581,NaN,NaN
851,NaN,NaN
865,NaN,NaN
880,NaN,NaN
889,NaN,NaN


In [16]:
hp_val[(hp_val['MasVnrType']=='None') & (hp_val['MasVnrArea']>0)][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea


In [17]:
# input mode for NaNs
nan_col_obj_list_fillna(hp_val)

In [18]:
# to visualize via dataframes the unique values that exist in the dataset
# to visualize the null count next to those values

nan_count(hp_val)

,Null_Count
LotFrontage,227


## Lot Frontage Imputation using KNN Imputer

In [19]:
# dummify the variables
hp_val_dum = pd.get_dummies(hp_val, drop_first=True)
hp_val_dum

# changing MSSubClass dtype so that it will not come out as float when naming columns
# hp_val_dum = hp_val_dum.astype({'MSSubClass': int})

# one hot encoding for mssubclass since it is numeric
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(hp_val_dum[['MSSubClass']])
oh_labels = enc.transform(hp_val_dum[['MSSubClass']]).toarray()
mssubcl_val_dum = pd.DataFrame(oh_labels)
mssubcl_val_dum.columns = enc.get_feature_names(['MSSubClass'])

# Concatenate the dataframes and drop Id and original MSSubClass
hp_val_dum = pd.concat([hp_val_dum, mssubcl_val_dum], axis=1, sort=False)
hp_val_dum = hp_val_dum.drop(['Id', 'MSSubClass'], axis=1)


'''
we are removing Id and MSSubclass
    - Id because it has no bearing on the result
    - MSSubClass because it is categorical
'''
# get numeric columns
cols_num = [x for x in hp_val.columns if hp_val[x].dtype!='object']
# not including the following
to_remove = ['Id', 'MSSubClass', 'LotFrontage']
cols_num_val_dum = [x for x in cols_num if x not in to_remove]

# part of the dataset to remove
hp_val_to_remove = hp_val[cols_num_val_dum]
# drop features to prevent duplicates
hp_val_dum = hp_val_dum.drop(cols_num_val_dum, axis=1)

# normalize the numeric columns
normaliz = sklearn.preprocessing.normalize(hp_val_to_remove, axis=0)
hp_num_val_norm_unf = pd.DataFrame(normaliz, columns=cols_num_val_dum)

# concatenate dummified obj cols and normalized numeric cols
hp_val_norm_unf = pd.concat([hp_num_val_norm_unf, hp_val_dum], axis=1)
hp_val_norm_unf.to_csv('..\hp_val_norm_unf.csv')

hp_val_norm_unf

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold,LotFrontage,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_20.0,MSSubClass_30.0,MSSubClass_40.0,MSSubClass_45.0,MSSubClass_50.0,MSSubClass_60.0,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_150.0,MSSubClass_160.0,MSSubClass_180.0,MSSubClass_190.0
0,0.027666,0.020957,0.027732,0.026040,0.025880,0.000000,0.019375,0.020455,0.010015,0.020328,0.019178,0.000000,0.0,0.015005,0.000000,0.000000,0.015713,0.000000,0.017617,0.02

- importing hp_val_norm after the segment in HousingPrice_LotFrontage_Imputation_DataSetManagement to add columns that were in the train dataset that were not found in the validation set

In [20]:
hp_val_norm = pd.read_csv('..\hp_val_norm.csv', index_col=0)
hp_val_norm

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold,LotFrontage,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_20.0,MSSubClass_30.0,MSSubClass_40.0,MSSubClass_45.0,MSSubClass_50.0,MSSubClass_60.0,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_150.0,MSSubClass_160.0,MSSubClass_180.0,MSSubClass_190.0,Utilities_NoSeWa,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,HouseStyle_2.5Fin,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_ImStucc,Exterior1st_Stone,Exter

### Running KNN Imputer then Normalize 'LotFrontage'

In [21]:
# run knn imputer
root_n = int(np.sqrt(len(hp_val_norm.columns)))
knn = KNNImputer(n_neighbors=root_n)

# we can run KNN Imputer on the whole dataset bc there are no null values in the other columns
hp_norm = pd.read_csv('..\hp_norm.csv', index_col=0)
knn_lf_norm = knn.fit(hp_norm)
knn_lf_val = knn.transform(hp_val_norm)

hp_val_strd = pd.DataFrame(knn_lf_val, columns=hp_val_norm.columns)

hp_val_strd.isna().values.any()

# Normalize 'LotFrontage'
lf_hp_val = hp_val_strd[['LotFrontage']]
hp_val_strd_drop = hp_val_strd.drop(['LotFrontage'], axis=1)
normaliz_lf_val = sklearn.preprocessing.normalize(lf_hp_val, axis=0)
normaliz_lf_val_df = pd.DataFrame(normaliz_lf_val, columns=['LotFrontage'])

hp_val_strd = pd.concat([hp_val_strd_drop, normaliz_lf_val_df], axis=1)
hp_val_strd.to_csv('..\hp_val_strd.csv')
hp_val_strd

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_20.0,MSSubClass_30.0,MSSubClass_40.0,MSSubClass_45.0,MSSubClass_50.0,MSSubClass_60.0,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_150.0,MSSubClass_160.0,MSSubClass_180.0,MSSubClass_190.0,Utilities_NoSeWa,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,HouseStyle_2.5Fin,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,Exterior1st_ImStucc,Exterior1st_Stone,Exterior2nd_Other

# Running Predictions

### MLR

In [22]:
# loading datasets
hp_full_strd = pd.read_csv('..\hp_full_strd.csv', index_col=0)
hp_lasso_strd_reduced = pd.read_csv('..\hp_lasso_strd_reduced.csv', index_col=0)
hp_logsaleprice = pd.read_csv('..\hp_logsaleprice.csv', index_col=0)

# dropping only for full and logsaleprice because hp_lasso_strd_reduced already has 820 dropped
hp_full_strd = hp_full_strd.drop([820])
hp_logsaleprice = hp_logsaleprice.drop([820])

In [23]:
lasso_cols = list(hp_lasso_strd_reduced.columns)
hp_val_lasso = hp_val_strd[lasso_cols]
hp_val_lasso

,LotArea,OverallQual,OverallCond,YearBuilt,MasVnrArea,BsmtFinSF1,TotalBsmtSF,1stFlrSF,GrLivArea,BsmtFullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,ScreenPorch,TotalBaths,BsmtFinTotalSF,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold,MSZoning_FV,MSZoning_RL,Street_Pave,LotShape_IR2,LotConfig_CulDSac,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Veenker,Condition1_Norm,Condition1_RRAe,BldgType_Duplex,BldgType_Twnhs,RoofStyle_Gable,RoofMatl_WdShngl,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_HdBoard,Exterior1st_VinylSd,Exterior1st_Wd Sdng,MasVnrType_Stone,ExterQual_Fa,ExterQual_TA,ExterCond_Fa,ExterCond_TA,Foundation_PConc,Foundation_Wood,BsmtQual_TA,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_No,BsmtFinType1_GLQ,BsmtFinType1_Rec,BsmtFinType2_BLQ,HeatingQC_Gd,HeatingQC_TA,CentralAir_Y,KitchenQual_TA,Functional_Maj2,Functional_Sev,Functional_Typ,GarageType_Basment,GarageType_CarPort,GarageFinish_Unf,GarageQual_Gd,GarageCond_Fa,PavedDrive_Y,SaleType_CWD,SaleType_ConLI,SaleType_New,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_30.0,MSSubClass_70.0,MSSubClass_90.0,MSSubClass_160.0,LotFrontage
0,0.027666,0.020957,0.027732,0.026040,0.000000,0.019375,0.020328,0.019178,0.015005,0.000000,0.000000,0.024626,0.019952,0.000000,0.013573,0.036742,0.023186,0.000000,0.053151,0.013957,0.023358,0.0,0.021905,0.030949,0.037878,0.040022,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.028514
1,0.033962,0.025148,0.027732,0.026000,0.013897,0.038213,0.030631,0.028447,0.022256,0.000000,0.041631,0.024626,0.023942,0.000000,0.013573,0.015704,0.065086,0.011204,0.000000,0.020936,0.035228,0.0,0.006572,0.030949,0.037878,0.042472,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.028870
2,0.032922,0.020957,0.023110,0.026518,0.000000,0.032748,0.021389,0.019863,0.027280,0.000000,0.041631,0.024626,0.023942,0.030096,0.027146,0.024260,0.035110,0.010582,0.000000,0.034893,0.030190,0.0,0.006207,0.030949,0.037878,0.009801,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.026375
3,0.023752,0.025148,0.027732,0.026531,0.002574,0.024923,0.021343,0.019821,0.026862,0.000000,0.041631,0.024626,0.027932,0.030096,0.027146,0.023656,0.059621,0.011204,0.000000,0.034893,0.022977,0.0,0.006572,0.030949,0.037878,0.009801,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.027801
4,0.011914,0.033531,0.023110,0.026451,0.000000,0.010888,0.029502,0.027398,0.021436,0.000000,0.000000,0.024626,0.019952,0.000000,0.027146,0.025468,0.000000,0.025521,0.063781,0.027915,0.010038,0.0,0.041255,0.030949,0.000000,0.014702,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.015326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [24]:
# multiple linear regression using lasso reduced dataset
xtrain_mlr, xtest_mlr, ytrain_mlr, ytest_mlr = ms.train_test_split(hp_lasso_strd_reduced,
                                                                   hp_logsaleprice,
                                                                   test_size=0.2,
                                                                   random_state=0)

# Must flatten to fit
ytrain_mlr = ytrain_mlr.values.flatten()

lm = LinearRegression()

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=0, shuffle=True)

# parameters to use
fitintercept = [True]
gparam_mlr = {'fit_intercept': fitintercept}

# setting parameters
gs_mlr = ms.GridSearchCV(lm, gparam_mlr, cv=n_folds, refit=True, n_jobs=-1,
                      scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_mlr.fit(xtrain_mlr, ytrain_mlr)

gs_mlr.predict(hp_val_lasso)

Wall time: 1.49 s


array([11.67392373, 11.97858138, 12.11187595, ..., 12.08097263,
       11.71397497, 12.34395001])

In [25]:
# create dataframe from array of predictions
prediction_mlr = pd.DataFrame([gs_mlr.predict(hp_val_lasso)])

# transpose dataframe as it will come out as 1 long row
prediction_mlr = prediction_mlr.T

# reset the index to add a column with index values to add the ID numbers
prediction_mlr = prediction_mlr.reset_index()
# add 1461 to the Id as resetting index will give us a column starting from 0
prediction_mlr['index'] = prediction_mlr['index'] + 1461
prediction_mlr.columns = ['Id', 'SalePrice']

# convert from log back to numeric
prediction_mlr['SalePrice'] = np.exp(prediction_mlr.SalePrice)

# convert to csv
prediction_mlr.to_csv('PREDICTIONS_MLR.csv', index=False, encoding='utf-8')
prediction_mlr

,Id,SalePrice
0,1461,117468.292588
1,1462,159305.875900
2,1463,182020.746804
3,1464,201434.950986
4,1465,197342.190078
...,...,...
1454,2915,84293.194255
1455,2916,76545.437409
1456,2917,176481.728538
1457,2918,122268.529932


## Lasso

In [26]:
# setting up train_full and test_full sets
xtrain_full, xtest_full, ytrain_full, ytest_full = ms.train_test_split(hp_full_strd,
                                                   hp_logsaleprice,
                                                   test_size=0.2,
                                                   random_state=0)

# Must flatten to fit
ytrain_full = ytrain_full.values.flatten()

lasso = Lasso(random_state=0, max_iter=1000000, selection='random',
              normalize=True, fit_intercept=True)

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=0, shuffle=True)

# parameters to use
alphas = np.linspace(1e-4, 2.5e-4, 30)
gparam_lasso_1 = {'alpha': alphas}

# setting parameters
gs_lasso_1 = ms.GridSearchCV(lasso, gparam_lasso_1, cv=n_folds, refit=True, n_jobs=-1,
                      scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_lasso_1.fit(xtrain_full, ytrain_full)

gs_lasso_1.predict(hp_val_strd)

Wall time: 2.34 s


array([11.35872855, 11.76577319, 11.63850665, ..., 11.53839886,
       11.37006954, 11.99070817])

In [27]:
# create dataframe from array of predictions
prediction_lasso = pd.DataFrame([gs_lasso_1.predict(hp_val_strd)])

# transpose dataframe as it will come out as 1 long row
prediction_lasso = prediction_lasso.T

# reset the index to add a column with index values to add the ID numbers
prediction_lasso = prediction_lasso.reset_index()
# add 1461 to the Id as resetting index will give us a column starting from 0
prediction_lasso['index'] = prediction_lasso['index'] + 1461
prediction_lasso.columns = ['Id', 'SalePrice']

# convert from log back to numeric
prediction_lasso['SalePrice'] = np.exp(prediction_lasso.SalePrice)

# convert to csv
prediction_lasso.to_csv('PREDICTIONS_LASSO.csv', index=False, encoding='utf-8')
prediction_lasso

,Id,SalePrice
0,1461,85710.322974
1,1462,128768.717454
2,1463,113380.721222
3,1464,119800.541295
4,1465,148100.768058
...,...,...
1454,2915,80809.144162
1455,2916,79296.900897
1456,2917,102580.061797
1457,2918,86687.895754


## Random Forest w Full Dataset

In [28]:
# setting up train_full and test_full sets
xtrain_rfr_strd, xtest_rfr_strd, ytrain_rfr_strd, ytest_rfr_strd = ms.train_test_split(hp_full_strd,
                                                                                   hp_logsaleprice,
                                                                                   test_size=0.2,
                                                                                   random_state=0)

# Must flatten to fit
ytrain_rfr_strd = ytrain_rfr_strd.values.flatten()

rfr = RandomForestRegressor()

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=0, shuffle=True)

# parameters to use
maxdepth = [481]
minsamlf = [2]
minsamsplt = [3]
nestimators = [487]
oobscore = [True]
warmstart = [False]

gparam_rfr = {'max_depth': maxdepth,
              'min_samples_leaf': minsamlf,
              'min_samples_split': minsamsplt,
              'n_estimators': nestimators,
              'oob_score': oobscore,
              'warm_start': warmstart}

# setting parameters
gs_rfr_strd = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                           scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_rfr_strd.fit(xtrain_rfr_strd, ytrain_rfr_strd)

gs_rfr_strd.predict(hp_val_strd)

Wall time: 12.5 s


array([11.67244169, 11.87517765, 12.12442825, ..., 11.92248491,
       11.57523118, 12.33781057])

In [29]:
# create dataframe from array of predictions
prediction_rfr_strd = pd.DataFrame([gs_rfr_strd.predict(hp_val_strd)])

# transpose dataframe as it will come out as 1 long row
prediction_rfr_strd = prediction_rfr_strd.T

# reset the index to add a column with index values to add the ID numbers
prediction_rfr_strd = prediction_rfr_strd.reset_index()
prediction_rfr_strd['index'] = prediction_rfr_strd['index'] + 1461

# rename columns
prediction_rfr_strd.columns = ['Id', 'SalePrice']

# convert SalePrice back to non-log float
prediction_rfr_strd['SalePrice'] = np.exp(prediction_rfr_strd['SalePrice'])

# convert to csv
prediction_rfr_strd.to_csv('PREDICTIONS_RFR_strd.csv', index=False, encoding='utf-8')

prediction_rfr_strd

,Id,SalePrice
0,1461,117294.329635
1,1462,143656.117564
2,1463,184319.925521
3,1464,181926.733475
4,1465,188589.472933
...,...,...
1454,2915,80183.701524
1455,2916,80391.255018
1456,2917,150615.409937
1457,2918,106428.766266


## Random Forest w Lasso Dataset

In [30]:
# setting up train_full and test_full sets
xtrain_rfr_lasso, xtest_rfr_lasso, ytrain_rfr_lasso, ytest_rfr_lasso = ms.train_test_split(hp_lasso_strd_reduced,
                                                                                   hp_logsaleprice,
                                                                                   test_size=0.2,
                                                                                   random_state=0)

# Must flatten to fit
ytrain_rfr_lasso = ytrain_rfr_lasso.values.flatten()

rfr = RandomForestRegressor()

# using standard kfold split
n_folds = ms.KFold(n_splits=5, random_state=0, shuffle=True)

# parameters to use
maxdepth = [50]
minsamlf = [2]
minsamsplt = [3]
nestimators = [250]
oobscore = [True]
warmstart = [False]

gparam_rfr = {'max_depth': maxdepth,
              'min_samples_leaf': minsamlf,
              'min_samples_split': minsamsplt,
              'n_estimators': nestimators,
              'oob_score': oobscore,
              'warm_start': warmstart}

# setting parameters
gs_rfr_lasso = ms.GridSearchCV(rfr, gparam_rfr, cv=n_folds, refit=True, n_jobs=-1,
                           scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_rfr_lasso.fit(xtrain_rfr_lasso, ytrain_rfr_lasso)

gs_rfr_lasso.predict(hp_val_lasso)

Wall time: 3.69 s


array([11.72965296, 11.92394788, 12.14996335, ..., 11.99438107,
       11.60927254, 12.35742394])

In [31]:
# create dataframe from array of predictions
prediction_rfr_lasso = pd.DataFrame([gs_rfr_lasso.predict(hp_val_lasso)])

# transpose dataframe as it will come out as 1 long row
prediction_rfr_lasso = prediction_rfr_lasso.T

# reset the index to add a column with index values to add the ID numbers
prediction_rfr_lasso = prediction_rfr_lasso.reset_index()
prediction_rfr_lasso['index'] = prediction_rfr_lasso['index'] + 1461

# rename columns
prediction_rfr_lasso.columns = ['Id', 'SalePrice']

# convert SalePrice back to non-log float
prediction_rfr_lasso['SalePrice'] = np.exp(prediction_rfr_lasso['SalePrice'])

# convert to csv
prediction_rfr_lasso.to_csv('PREDICTIONS_RFR_lasso.csv', index=False, encoding='utf-8')

prediction_rfr_lasso

,Id,SalePrice
0,1461,124200.560259
1,1462,150835.916604
2,1463,189087.158816
3,1464,186995.574420
4,1465,191441.711862
...,...,...
1454,2915,87969.417986
1455,2916,86856.382142
1456,2917,161842.847586
1457,2918,110114.117428


## Gradient Boosting w Full Dataset

In [32]:
# after seeing that removing the outlier causes a worse test score, will reset the data to include it
hp_full_strd = pd.read_csv('..\hp_full_strd.csv', index_col=0)
hp_logsaleprice = pd.read_csv('..\hp_logsaleprice.csv', index_col=0)

state = 0

xtrain_gb_full, xtest_gb_full, ytrain_gb_full, ytest_gb_full = ms.train_test_split(hp_full_strd,
                                                                               hp_logsaleprice,
                                                                               test_size=0.2,
                                                                               random_state=0)

ytrain_gb_full = ytrain_gb_full.values.flatten()

gbm = GradientBoostingRegressor(verbose=1)

n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

In [33]:
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

learningrate = [0.041]
nestimators = [392]
maxfeatures = [21]
maxdepth = [4]

gparam_gbm = {'learning_rate': learningrate,
              'n_estimators': nestimators, 
              'max_features': maxfeatures,
              'max_depth': maxdepth}

gs_gb_full = GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1, 
                      scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_gb_full.fit(xtrain_gb_full, ytrain_gb_full)

gs_gb_full.predict(hp_val_strd)

      Iter       Train Loss   Remaining Time 
         1           0.1493            0.78s
         2           0.1410            0.39s
         3           0.1330            0.52s
         4           0.1263            0.39s
         5           0.1198            0.39s
         6           0.1131            0.45s
         7           0.1073            0.44s
         8           0.1017            0.43s
         9           0.0968            0.43s
        10           0.0923            0.42s
        20           0.0580            0.35s
        30           0.0385            0.34s
        40           0.0273            0.33s
        50           0.0204            0.31s
        60           0.0163            0.30s
        70           0.0135            0.29s
        80           0.0117            0.28s
        90           0.0104            0.27s
       100           0.0093            0.26s
       200           0.0054            0.17s
       300           0.0039            0.08s
Wall time

array([11.64455665, 11.9610192 , 11.95184255, ..., 11.90532342,
       11.58571838, 12.22921486])

In [34]:
# create dataframe from array of predictions
prediction_gb_full = pd.DataFrame([gs_gb_full.predict(hp_val_strd)])

# transpose dataframe as it will come out as 1 long row
prediction_gb_full = prediction_gb_full.T

# reset the index to add a column with index values to add the ID numbers
prediction_gb_full = prediction_gb_full.reset_index()
prediction_gb_full['index'] = prediction_gb_full['index'] + 1461

# rename columns
prediction_gb_full.columns = ['Id', 'SalePrice']

# convert SalePrice back to non-log float
prediction_gb_full['SalePrice'] = np.exp(prediction_gb_full['SalePrice'])

# convert to csv
prediction_gb_full.to_csv('PREDICTIONS_gb_full.csv', index=False, encoding='utf-8')

prediction_gb_full

,Id,SalePrice
0,1461,114068.754218
1,1462,156532.541898
2,1463,155102.667218
3,1464,153072.579870
4,1465,161348.555658
...,...,...
1454,2915,72534.002399
1455,2916,78732.045122
1456,2917,148052.677860
1457,2918,107550.779257


## Gradient Boosting w Reduced Dataset

In [35]:
hp_rfr_fltr = pd.read_csv('..\hp_rfr_fltr.csv', index_col=0)
hp_val_rfr = hp_val_strd[hp_rfr_fltr.columns]
hp_val_rfr

,OverallQual,GrLivArea,TotalBsmtSF,GarageCars,GarageArea,1stFlrSF,YearBuilt,BsmtFinSF1,BsmtFinTotalSF,CentralAir_Y,OverallCond,LotArea,YrsRemodToSold,Fireplaces,LotFrontage,YearRemodAdd,GarageFinish_Unf,2ndFlrSF,MSZoning_RM,BsmtUnfSF,GarageType_Detchd,GarageYrBlt,TotalBaths,MasVnrArea,TotalPorchSF,TotRmsAbvGrd,MoSold,OpenPorchSF,WoodDeckSF,ExterCond_Fa,KitchenAbvGr,LotShape_Reg,GarageType_Attchd,BsmtQual_Gd,KitchenQual_Gd,MSSubClass_30.0,YrSold,KitchenQual_TA,SaleCondition_Normal,EnclosedPorch,BedroomAbvGr,PavedDrive_Y,Neighborhood_IDOTRR,MSZoning_RL,BsmtBaths,HeatingQC_TA,Electrical_SBrkr,ExterQual_TA,Neighborhood_OldTown,BsmtQual_TA,BsmtExposure_No,BsmtExposure_Gd,HeatingQC_Fa,HalfBath,Functional_Typ,Condition1_Norm,BsmtFinType1_GLQ,SaleCondition_Family,GarageFinish_RFn,BsmtFinType1_Unf,Neighborhood_NAmes,RoofStyle_Hip,Neighborhood_Edwards,Exterior1st_VinylSd,FullBath,Neighborhood_Crawfor,ExterCond_TA,HeatingQC_Gd,ExterQual_Gd,LandContour_Lvl,RoofStyle_Gable,Foundation_CBlock,MSSubClass_60.0,LotConfig_Inside,Exterior2nd_VinylSd,Exterior1st_MetalSd,FireplaceQu_Gd,ScreenPorch,BsmtFullBath,Foundation_PConc,BsmtCond_TA,Exterior1st_Wd Sdng,LandSlope_Mod,SaleType_WD,Exterior2nd_MetalSd,ExterQual_Fa,MSSubClass_50.0,Porches,Decks,Exterior1st_BrkFace,Neighborhood_NWAmes,MSSubClass_20.0,Condition1_Feedr,BsmtFinType1_Rec,Exterior2nd_Plywood,ExterCond_Gd,Neighborhood_Sawyer,BsmtHalfBath,BsmtFinSF2,HouseStyle_2Story,HouseStyle_1Story,FireplaceQu_TA,FireplaceQu_Po,MasVnrType_BrkFace,SaleCondition_Partial,GarageType_CarPort,KitchenQual_Fa,Neighborhood_CollgCr,Exterior2nd_Wd Sdng,GarageQual_Fa,Neighborhood_Somerst,Exterior2nd_HdBoard,BsmtQual_Fa,SaleType_New,Heating_Grav,Neighborhood_BrkSide,Exterior1st_Plywood,Exterior1st_HdBoard,MasVnrType_Stone,MiscVal,BsmtFinType1_BLQ,MasVnrType_None,LandContour_HLS,MSSubClass_70.0,GarageCond_Fa,PavedDrive_P,Neighborhood_NoRidge,BsmtExposure_Mn,LotConfig_CulDSac,GarageQual_TA,MSSubClass_80.0,2ndFlr,BsmtFinType2_BLQ,BsmtFinType1_LwQ,Functional_Maj2,Functional_Mod,BsmtUnfin,MSSubClass_160.0,Neighborhood_ClearCr,BldgType_Duplex,Exterior2nd_CmentBd,Functional_Min2,LowQualFinSF,GarageCond_TA,Neighborhood_MeadowV,Heating_GasA,Neighborhood_NridgHt,Neighborhood_StoneBr,BsmtFinType2_Unf,LotShape_IR2,Electrical_FuseF,LandContour_Low,Exterior2nd_Stucco,Neighborhood_Mitchel,GarageType_BuiltIn,RoofStyle_Gambrel,Foundation_Slab,Neighborhood_Gilbert,Exterior1st_CemntBd,Neighborhood_Timber,MSZoning_FV,HouseStyle_SLvl,Functional_Min1,Exterior2nd_Wd Shng,MSSubClass_90.0,Neighborhood_SWISU,Condition2_Norm,BldgType_TwnhsE,SaleType_ConLD,LowQualFin,BsmtFinType2_Rec,3SsnPorch,MSSubClass_120.0,BsmtCond_Gd,BldgType_2fmCon,BldgType_Twnhs,RoofMatl_Tar&Grv,Exterior2nd_BrkFace,MSSubClass_190.0,Neighborhood_SawyerW,HouseStyle_1.5Unf,Street_Pave,FireplaceQu_Fa,BsmtFinType2_LwQ,HouseStyle_2.5Unf,Condition1_PosN,GarageType_Basment,Condition1_RRAe,GarageQual_Gd,Exterior2nd_Brk Cmn,MSSubClass_180.0,MSZoning_RH,LotConfig_FR2,Neighborhood_Veenker,Exterior1st_WdShing,GarageCond_Po,Condition1_RRAn,HouseStyle_SFoyer,SaleCondition_Alloca,Exterior1st_Stucco,MSSubClass_75.0,Neighborhood_NPkVill,BsmtFinType2_GLQ,LandSlope_Sev,Neighborhood_BrDale,MSSubClass_85.0,MSSubClass_45.0,Condition1_PosA,LotShape_IR3,GarageQual_Po,Heating_GasW,Heating_Wall,RoofStyle_Mansard,SaleType_ConLw,Foundation_Stone,Exterior2nd_Stone,GarageCond_Gd
0,0.020957,0.015005,0.020328,0.013573,0.036742,0.019178,0.026040,0.019375,0.023358,1.0,0.027732,0.027666,0.040022,0.000000,0.028514,0.025880,1.0,0.000000,0.0,0.010015,0.0,0.025957,0.013957,0.000000,0.021905,0.019952,0.023503,0.000000,0.023186,0.0,0.024626,1.0,1.0,0.0,0.0,0.0,0.026209,1.0,1.0,0.0,0.017617,1.0,0.0,0.0,0.000000,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.015713,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.053151,0.000000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.030949,0.037878,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.000000,0.020455,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [36]:
state = 0

xtrain_rfr_fltr, xtest_rfr_fltr, ytrain_rfr_fltr, ytest_rfr_fltr = ms.train_test_split(hp_rfr_fltr,
                                                                               hp_logsaleprice,
                                                                               test_size=0.2,
                                                                               random_state=state)

ytrain_rfr_fltr = ytrain_rfr_fltr.values.flatten()

gbm = GradientBoostingRegressor(verbose=1)

n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

In [37]:
learningrate = [0.08]
nestimators = [188]
maxfeatures = [24]
maxdepth = [4]

gparam_gbm = {'learning_rate': learningrate,
              'n_estimators': nestimators, 
              'max_features': maxfeatures,
              'max_depth': maxdepth}

gs_gbm_rfrfltr = GridSearchCV(gbm, gparam_gbm, cv=n_folds, refit=True, n_jobs=-1, 
                      scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_gbm_rfrfltr.fit(xtrain_rfr_fltr, ytrain_rfr_fltr)

gs_gbm_rfrfltr.predict(hp_val_rfr)

      Iter       Train Loss   Remaining Time 
         1           0.1399            0.19s
         2           0.1257            0.28s
         3           0.1126            0.25s
         4           0.1022            0.23s
         5           0.0927            0.22s
         6           0.0837            0.24s
         7           0.0759            0.23s
         8           0.0691            0.23s
         9           0.0625            0.22s
        10           0.0571            0.21s
        20           0.0270            0.19s
        30           0.0164            0.17s
        40           0.0119            0.16s
        50           0.0095            0.15s
        60           0.0082            0.14s
        70           0.0073            0.13s
        80           0.0065            0.12s
        90           0.0058            0.11s
       100           0.0052            0.09s
Wall time: 531 ms


array([11.80076848, 11.9392122 , 12.12070427, ..., 12.04803849,
       11.64160855, 12.3488473 ])

In [38]:
# create dataframe from array of predictions
prediction_gbm_rfrfltr = pd.DataFrame([gs_gbm_rfrfltr.predict(hp_val_rfr)])

# transpose dataframe as it will come out as 1 long row
prediction_gbm_rfrfltr = prediction_gbm_rfrfltr.T

# reset the index to add a column with index values to add the ID numbers
prediction_gbm_rfrfltr = prediction_gbm_rfrfltr.reset_index()
prediction_gbm_rfrfltr['index'] = prediction_gbm_rfrfltr['index'] + 1461

# rename columns
prediction_gbm_rfrfltr.columns = ['Id', 'SalePrice']

# convert SalePrice back to non-log float
prediction_gbm_rfrfltr['SalePrice'] = np.exp(prediction_gbm_rfrfltr['SalePrice'])

# convert to csv
prediction_gbm_rfrfltr.to_csv('PREDICTIONS_gbm_rfrfltr.csv', index=False, encoding='utf-8')

prediction_gbm_rfrfltr

,Id,SalePrice
0,1461,133354.793741
1,1462,153155.986625
2,1463,183634.797485
3,1464,206542.406334
4,1465,192154.584221
...,...,...
1454,2915,82299.691234
1455,2916,82758.054879
1456,2917,170764.124253
1457,2918,113732.963592


# Attempt with XGBoost

In [39]:
hp_full_strd = pd.read_csv('..\hp_full_strd.csv', index_col=0)
hp_full_strd

,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalBaths,BsmtBaths,BsmtFinTotalSF,BsmtUnfin,2ndFlr,LowQualFin,TotalPorchSF,Porches,Decks,YrsRemodToSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_20.0,MSSubClass_30.0,MSSubClass_40.0,MSSubClass_45.0,MSSubClass_50.0,MSSubClass_60.0,MSSubClass_70.0,MSSubClass_75.0,MSSubClass_80.0,MSSubClass_85.0,MSSubClass_90.0,MSSubClass_120.0,MSSubClass_1

In [40]:
# train test split using the dataframe from above
state = 0

xtrain_xgb_full, xtest_xgb_full, ytrain_xgb_full, ytest_xgb_full = ms.train_test_split(hp_full_strd,
                                                   hp_logsaleprice,
                                                   test_size=0.2,
                                                   random_state=state)

ytrain_xgb_full = ytrain_xgb_full.values.flatten()
n_folds = ms.KFold(n_splits=5, random_state=state, shuffle=True)

# using GPU
xg_reg = xgb.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist',
                          predictor='gpu_predictor', random_state=state,
                          eval_metric='rmse', gamma=0.001)

In [46]:
# from https://stackoverflow.com/questions/25878198/change-pandas-dataframe-column-order-in-place
'''
this was done in order to have the validation set
columns be in the same order for xgboost prediction
'''
col_order = hp_full_strd.columns

for column in col_order:
    hp_val_strd[column] = hp_val_strd.pop(column)

In [49]:
# using lists to store values for dataframe

# setting parameters
nestimators = [490]
maxdepth = [2]
learningrate = [0.125]


gparam_xgb = {'n_estimators': nestimators,
              'max_depth': maxdepth,
              'learning_rate': learningrate}

# applying gridsearch using the n_folds value from above
gs_xgb_full = ms.GridSearchCV(xg_reg, gparam_xgb, cv=n_folds, refit=True, n_jobs=-1,
                      scoring='neg_root_mean_squared_error', return_train_score=True)

%time gs_xgb_full.fit(xtrain_xgb_full, ytrain_xgb_full)

gs_xgb_full.predict(hp_val_strd)

Wall time: 9.65 s


array([11.6553955, 11.92014  , 12.061697 , ..., 11.961674 , 11.660841 ,
       12.220559 ], dtype=float32)

In [50]:
# create dataframe from array of predictions
prediction_xgb_full = pd.DataFrame([gs_xgb_full.predict(hp_val_strd)])

# transpose dataframe as it will come out as 1 long row
prediction_xgb_full = prediction_xgb_full.T

# reset the index to add a column with index values to add the ID numbers
prediction_xgb_full = prediction_xgb_full.reset_index()
prediction_xgb_full['index'] = prediction_xgb_full['index'] + 1461

# rename columns
prediction_xgb_full.columns = ['Id', 'SalePrice']

# convert SalePrice back to non-log float
prediction_xgb_full['SalePrice'] = np.exp(prediction_xgb_full['SalePrice'])

# convert to csv
prediction_xgb_full.to_csv('PREDICTIONS_xgb_full.csv', index=False, encoding='utf-8')

prediction_xgb_full

,Id,SalePrice
0,1461,115311.853629
1,1462,150262.683742
2,1463,173112.509534
3,1464,184995.725324
4,1465,172811.641742
...,...,...
1454,2915,71341.593447
1455,2916,72448.683310
1456,2917,156635.031271
1457,2918,115941.494855
